# Курсовая работа
## Выполнил студент группы БФИ2203 Закаморный Михаил

Написать программу, которая будет считывать данные из CSV файла, содержащего
информацию о продажах товаров в магазине. Данные в файле содержатся в следующем
формате:

| Номер заказа | Дата заказа | Название товара | Категория товара | Количество продаж | Цена
за единицу | Общая стоимость |

Необходимо:
1. Рассчитать общую выручку магазина.
2. Найти товар, который был продан наибольшее количество раз.
3. Найти товар, который принес наибольшую выручку.
4. Составить отчет, содержащий информацию об общей выручке магазина, количестве
проданных единиц каждого товар и доле каждого товара в общей выручке.
Для решения задач необходимо использовать структуры данных, такие как массивы и
хеш-таблицы, а также различные алгоритмы обработки данных, например, сортировку и поиск.
Также необходимо учитывать возможные ошибки ввода-вывода и обрабатывать их в
соответствии с требованиями.

# Ход работы:

Создадим класс хэш-таблицы:

In [2]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.table = [[] for _ in range(size)]

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        index = self.hash_function(key)
        for pair in self.table[index]:
            if pair[0] == key:
                pair[1] = value
                return
        self.table[index].append([key, value])

    def get(self, key):
        index = self.hash_function(key)
        for pair in self.table[index]:
            if pair[0] == key:
                return pair[1]
        return None

    def items(self):
        for bucket in self.table:
            for pair in bucket:
                yield pair

Создадим функцию сортировки Шелла:

In [3]:
def shell_sort(arr, key):
    n = len(arr)
    gap = n // 2
    while gap > 0:
        for i in range(gap, n):
            temp = arr[i]
            j = i
            while j >= gap and key(arr[j - gap]) > key(temp):
                arr[j] = arr[j - gap]
                j -= gap
            arr[j] = temp
        gap //= 2

Создадим функцию для загрузки хэш-таблицы:

In [4]:
import csv
def parse_csv(file_path, delimiter=';', encoding='utf-8'):
    with open(file_path, newline='', encoding=encoding) as csvfile:
        reader = csv.DictReader(csvfile, delimiter=delimiter)
        data = []
        for row in reader:
            data.append(row)
    return data

Функция для расчёта:
- Общей выручки
- Товара с наибольшим кол-вом продаж
- Товара с наибольшей выручкой

In [5]:
def calculate_total_revenue(data):
    total_revenue = sum(float(row['Total cost']) for row in data)
    return total_revenue

def find_most_sold_product(data):
    product_sales = HashTable(1000)
    for row in data:
        product = row['Product name']
        quantity = int(row['Quantity'])
        current_quantity = product_sales.get(product)
        if current_quantity is None:
            product_sales.insert(product, quantity)
        else:
            product_sales.insert(product, current_quantity + quantity)
    most_sold_product = None
    most_sold_quantity = 0
    for product, quantity in product_sales.items():
        if quantity > most_sold_quantity:
            most_sold_product = product
            most_sold_quantity = quantity
    return most_sold_product, most_sold_quantity

def find_highest_revenue_product(data):
    product_revenue = HashTable(1000)
    for row in data:
        product = row['Product name']
        revenue = float(row['Total cost'])
        current_revenue = product_revenue.get(product)
        if current_revenue is None:
            product_revenue.insert(product, revenue)
        else:
            product_revenue.insert(product, current_revenue + revenue)
    highest_revenue_product = None
    highest_revenue = 0.0
    for product, revenue in product_revenue.items():
        if revenue > highest_revenue:
            highest_revenue_product = product
            highest_revenue = revenue
    return highest_revenue_product, highest_revenue


Сделаем функцию для вывода результатов:

In [6]:
def generate_report(data):
    product_stats = HashTable(1000)
    total_revenue = calculate_total_revenue(data)
    for row in data:
        product = row['Product name']
        quantity = int(row['Quantity'])
        revenue = float(row['Total cost'])
        stats = product_stats.get(product)
        if stats is None:
            product_stats.insert(product, {'quantity': quantity, 'revenue': revenue})
        else:
            stats['quantity'] += quantity
            stats['revenue'] += revenue
            product_stats.insert(product, stats)

    report = []
    for product, stats in product_stats.items():
        percentage = (stats['revenue'] / total_revenue) * 100
        report.append({
            'Название товара': product,
            'Количество продаж': stats['quantity'],
            'Общая выручка': stats['revenue'],
            'Доля в общей выручке (%)': percentage
        })
    return report

Выведем данные по таблице:

In [7]:
data = None
try:
  file_path = 'input.csv'
  data = parse_csv(file_path)
  #Расчет общей выручки магазина
  total_revenue = calculate_total_revenue(data)
  print(f'\nОбщая выручка магазина: {int(total_revenue)} руб.')

  #Поиск товара, который был продан наибольшее количество раз
  most_sold_product, most_sold_quantity = find_most_sold_product(data)
  print(f'Товар, который был продан наибольшее количество раз: {most_sold_product} ({most_sold_quantity} шт.)')

  #Поиск товара, который принес наибольшую выручку
  highest_revenue_product, highest_revenue = find_highest_revenue_product(data)
  print(f'Товар, который принес наибольшую выручку: {highest_revenue_product} ({int(highest_revenue)} руб.)')

  #Составление отчета
  report = generate_report(data)

  # Сортировка Шелла для отчета по выручке 
  shell_sort(report, key=lambda x: x['Общая выручка'])


  print('\nОтчет по товарам:')
  print("—"*136)
  for item in reversed(report): #reversed чтобы общая выручка показывалась по убыванию
    print( '{:<47}'.format(item['Название товара']),
          "Количество продаж:",
          '{:>3}'.format(item['Количество продаж']),3*" ",
            f"Общая выручка:",
            '{:>7}'.format(round(item['Общая выручка'])),"руб.",3*" ",
            "Доля в общей выручке:",
            '{:>4}'.format(round(item['Доля в общей выручке (%)'],1)), "%")
    print("—"*136)
except FileNotFoundError:
  print('CSV-файл не найден!')




Общая выручка магазина: 13197834 руб.
Товар, который был продан наибольшее количество раз: MXR Patch Cable (111 шт.)
Товар, который принес наибольшую выручку: Fender Stratocaster Standard Made in Mexico (3923600 руб.)

Отчет по товарам:
————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Fender Stratocaster Standard Made in Mexico     Количество продаж:  34     Общая выручка: 3923600 руб.     Доля в общей выручке: 29.7 %
————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Fender Squier Stratocaster Standard             Количество продаж:  51     Общая выручка: 2850849 руб.     Доля в общей выручке: 21.6 %
————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Gibson Flying V                                 Количество продаж:   8     Обща

# Вывод:

Мы написали программу, которая считывает данные продаж магазина музыкальных инструментов и аксессуаров из CSV файла, рассчитали общую выручку магазина, нашли товар, который был продан наибольшее количество раз, товар, который принес наибольшую выручку, составили отчет, содержащий количество продаж товаров, их общую выручку и долю в общей выручке.
